# Precog Project 

In [1]:
#  I have a folder in which named TEST_dataset, which has 3 sub folder named namo, ak , and people 
#  containing 50 images in each of them

In [25]:
import sys
import os
from skimage import io
# import dlib
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import numpy as np
from PIL import Image
from scipy import ndimage
from six.moves import cPickle as pickle
np.random.seed(133)

In [26]:
# We now want to convert the entire dataset into a 3D array
# (image index, x, y), with mean = 0  and s.d. = 0.5
#  give 3 diff folderpaths namo, ak, people
img_wd = 200 #it is known
img_ht = 200 #it is known

def load_pictures(folderpath):
    '''Loading for a single image'''
    img_files = os.listdir(folderpath)
    tot_imgs = len(img_files)
    
#     print img_files[1]
#     full_path = os.path.join(folderpath,img_files[1])
#     img_data = (ndimage.imread(full_path).astype(float) - 255.0/2)/255.0
#     for converting (200,200,3)
#     img_data = img_data[:,:,0]
#     print img_data.shape
#     print img_data
#     new_arr = img_data.reshape(120000)
#     print new_arr
    
    dataset = np.ndarray(shape = (tot_imgs, img_wd*img_ht*3),
                         dtype = np.float32)
    img_index = 0
    for img in img_files:
        img_file_path = os.path.join(folderpath, img)
        try:
            img_data = (ndimage.imread(img_file_path).astype(float) - 255.0/2)/255.0
#             # We did the above to set the range of the image data to 
#             # -0.5 to 0.5 (s.d.)
            img_data = img_data.reshape(img_wd*img_ht*3)
            dataset[img_index, :] = img_data
            img_index = img_index + 1
        except IOError as e:
            print('file read error, skipping thsi file')
                
    print('Full dataset shape', dataset.shape)
    print('Dataset Mean', np.mean(dataset))
    print('Dataset SD', np.std(dataset))
    return dataset

In [6]:
data_set_ak = load_pictures("./TEST_dataset/ak")
data_set_namo = load_pictures("./TEST_dataset/namo")
data_set_people = load_pictures("./TEST_dataset/people")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


('Full dataset shape', (400, 120000))
('Dataset Mean', -0.065289773)
('Dataset SD', 0.24625809)
('Full dataset shape', (400, 120000))
('Dataset Mean', 0.0063913576)
('Dataset SD', 0.22900333)
('Full dataset shape', (400, 120000))
('Dataset Mean', -0.030012256)
('Dataset SD', 0.26703089)


In [7]:
# now i have created Flattened Image data set for each three 
# of categories

#  one such db can be 
# [ R,G,B,R,G,B,R,G,B,R,G,B,R,G,B,R,G,B ..... 200*200  ] pixals
# .
# .
# .
# .
# .
# for all images for a given person
# so now size of a given db is (400*20000)
#  next task to create a label for each dataset 
# if 1 represents modi
# it should be like
# [1]
# [1]
# [1]
# [1]
# CONVENTION
# 1 --> NAMO
# 2 --> AK
# 3 --> PEOPLE

In [8]:
# now I have to divide dataset in TRAIN, TEST, CROSS_VALIDATION
# total 400 images of each
# 50% --> TRAIN (300 imgs in each class)
# 25% -->TEST (50 imgs '' )
# 25% -->CV (50 imgs '')

In [27]:
dataset_train_namo = data_set_namo[0:300]
dataset_test_namo = data_set_namo[300:350]
dataset_cv_namo = data_set_namo[350:400]

dataset_train_ak = data_set_ak[0:300]
dataset_test_ak = data_set_ak[300:350]
dataset_cv_ak = data_set_ak[350:400]

dataset_train_people = data_set_people[0:300]
dataset_test_people = data_set_people[300:350]
dataset_cv_people = data_set_people[350:400]


In [28]:
dataset_train_final = np.concatenate((dataset_train_namo,dataset_train_ak))
dataset_train_final = np.concatenate((dataset_train_final,dataset_train_people))
print dataset_train_final.shape

dataset_test_final = np.concatenate((dataset_test_namo,dataset_test_ak))
dataset_test_final = np.concatenate((dataset_test_final,dataset_test_people))
print dataset_test_final.shape

dataset_cv_final = np.concatenate((dataset_cv_namo,dataset_cv_ak))
dataset_cv_final = np.concatenate((dataset_cv_final,dataset_cv_people))
print dataset_test_final.shape

(900, 120000)
(150, 120000)
(150, 120000)


In [29]:
# total row in train data set = 900

label_train = np.ndarray(900, dtype=np.int32)
label_train[0:300]=1 # for namo 
label_train[300:600]=2 # for ak
label_train[600:900]=3 # for people
print label_train.size

label_test = np.ndarray(150, dtype=np.int32)
label_test[0:50]=1 # for namo 
label_test[50:100]=2 # for ak
label_test[100:150]=3 # for people
print label_test.size


label_cv = np.ndarray(150, dtype=np.int32)
label_cv[0:50]=1 # for namo 
label_cv[50:100]=2 # for ak
label_cv[100:150]=3 # for people
print label_cv.size
#     just an other way to label, I was trying
# label_array_namo = np.full((400,1),1)
# label_array_ak = np.full((400,1),2)
# label_array_people = np.full((400,1),3)
# # print label_array_ak

900
150
150


In [19]:
# randomizing the database w.r.t. to labels
def randomize(dataset,labels,size):
    perm = np.random.permutation(size)
    shuffled_dataset = dataset[perm,:]
    shuffled_labels = labels[perm]
    return shuffled_dataset, shuffled_labels

In [30]:
dataset_train_final, label_train = randomize(dataset_train_final,label_train,900)
dataset_test_final, label_test = randomize(dataset_test_final,label_test,150)
dataset_cv_final, label_cv = randomize(dataset_cv_final,label_cv,150)

print dataset_train_final.shape,label_train.shape
print dataset_test_final.shape, label_test.shape
print dataset_cv_final.shape, label_cv.shape
# print label_test

(900, 120000) (900,)
(150, 120000) (150,)
(150, 120000) (150,)


In [23]:
# saving files to pickle

with open("./dataset_train_final.pickle",'wb') as f:
    pickle.dump(dataset_train_final,f, pickle.HIGHEST_PROTOCOL)

with open("./label_train.pickle",'wb') as f:
    pickle.dump(label_train,f, pickle.HIGHEST_PROTOCOL)

with open("./dataset_test_final.pickle",'wb') as f:
    pickle.dump(dataset_test_final,f, pickle.HIGHEST_PROTOCOL)

with open("./label_test.pickle",'wb') as f:
    pickle.dump(label_test,f, pickle.HIGHEST_PROTOCOL)

with open("./dataset_cv_final.pickle",'wb') as f:
    pickle.dump(dataset_cv_final,f, pickle.HIGHEST_PROTOCOL)

with open("./label_cv.pickle",'wb') as f:
    pickle.dump(label_cv,f, pickle.HIGHEST_PROTOCOL)


In [24]:
#  for fetching data from .pickle files
# with open("./dataset_train_final.pickle", 'rb') as f:
#     dataset_train_final = pickle.load(f)

# with open("./label_train.pickle", 'rb') as f:
#     label_train = pickle.load(f)

# with open("./dataset_test_final.pickle", 'rb') as f:
#     dataset_test_final = pickle.load(f)

# with open("./label_test.pickle", 'rb') as f:
#     label_test = pickle.load(f)

# with open("./dataset_cv_final.pickle", 'rb') as f:
#     dataset_cv_final = pickle.load(f)

# with open("./label_cv.pickle", 'rb') as f:
#     label_cv = pickle.load(f)

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [34]:
# There was a known issue in LR, so currently trying to avoid it
model = LogisticRegression()
print dataset_train_final.size
print label_train.shape

108000000
(900,)


In [36]:
model.fit(dataset_train_final,label_train)
predict_LR = model.predict(dataset_test_final)
print(accuracy_score(predict_LR,label_test))
# accuracy 0.92

0.92


In [ ]:
from sklearn import tree
gnb = tree.DecisionTreeClassifier()
gnb.fit(dataset_train_final,label_train)
predict_DT = gnb.predict(dataset_test_final)
print(accuracy_score(predict_DT,label_test))
# accuracy 0.78

In [ ]:
# classifier = SVC()
# classifier.fit(dataset_train_final,label_train)
# predicc = classifier.predict(dataset_test_final)
# print(accuracy_score(predicc,label_test))
# This gave me 0.3 accuracy

In [14]:
# def normalize(x):
#     #using formulae = (x-x(min))/(x(max)-x(min))
#     x_min = np.min(x)
#     x_max = np.max(x)
#     x_final_normalised = list()
# #     print "xmin ", x_min
# #     print "xmax ", x_max
    
#     for i in x:
# #         print i
#         x_final_normalised.append((i-x_min)/(x_max-x_min))
        
#     return np.array(x_final_normalised)
  

In [15]:
# normalised_array = normalize(image)

In [16]:
# def encode(x):
#     y = np.zeros(3)
#     np.put(y,x,1)
#     return y

In [17]:
# def one_hot_encode(x):
#     one_hot = list()
    
#     for i in x:
#         one_hot.append(encode(i))
#     return np.array(one_hot)

In [ ]:
# something to be done here, preprocess training,
# valid, and testing set
# preprossing includes, normalising, one_hot_encode

In [ ]:
# leave it above